In [118]:
from pathlib import Path
import sys
sys.path.append(str(Path('src').absolute()))
sys.path.append(str(Path('src/models').absolute()))
import models as m
from dataset_v5 import *
from Test_v2 import Test
from librosa import load, effects
import torch
import sounddevice as sd
import numpy as np

In [119]:
data_path = Path('data/test/qwe.mp3')

In [120]:
model = torch.load('weight/CNN_LSTM_5_128_200T20230613141052.pth')

In [121]:
model.to('cuda')

CNN_LSTM_5(
  (conv1): Conv1d(128, 32, kernel_size=(32,), stride=(1,), padding=(1,))
  (dropout1): Dropout(p=0.3, inplace=False)
  (conv2): Conv1d(32, 64, kernel_size=(18,), stride=(1,), padding=(1,))
  (dropout2): Dropout(p=0.3, inplace=False)
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (dropout3): Dropout(p=0.3, inplace=False)
  (lstm1): LSTM(128, 128, batch_first=True)
  (lstm2): LSTM(128, 128, batch_first=True)
  (dense1): Linear(in_features=128, out_features=64, bias=True)
  (dense2): Linear(in_features=64, out_features=32, bias=True)
  (dense3): Linear(in_features=32, out_features=16, bias=True)
  (dense4): Linear(in_features=16, out_features=8, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=32, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

In [122]:
model.eval()

CNN_LSTM_5(
  (conv1): Conv1d(128, 32, kernel_size=(32,), stride=(1,), padding=(1,))
  (dropout1): Dropout(p=0.3, inplace=False)
  (conv2): Conv1d(32, 64, kernel_size=(18,), stride=(1,), padding=(1,))
  (dropout2): Dropout(p=0.3, inplace=False)
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (dropout3): Dropout(p=0.3, inplace=False)
  (lstm1): LSTM(128, 128, batch_first=True)
  (lstm2): LSTM(128, 128, batch_first=True)
  (dense1): Linear(in_features=128, out_features=64, bias=True)
  (dense2): Linear(in_features=64, out_features=32, bias=True)
  (dense3): Linear(in_features=32, out_features=16, bias=True)
  (dense4): Linear(in_features=16, out_features=8, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=32, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

In [123]:
x, sr = load(data_path, duration=5, sr=44100, mono=True)

In [124]:
x.shape

(205330,)

In [125]:
from torchvision import transforms

In [126]:
duration = 5

In [127]:
stretch_rate = (x.shape[0] / sr) / duration

In [128]:
x_stretch = effects.time_stretch(y=x, rate=stretch_rate)

In [129]:
class ExpandDimsTransform:
    def __init__(self, axis=0):
        self.axis = axis

    def __call__(self, y):
        return np.expand_dims(y, axis=self.axis)

In [130]:
transform = transforms.Compose([
    PreEmphasisTransform(coef=0.97),
    MelSpectrogramTransform(sr=44100, n_fft=2048, hop_length=512, win_length=None, window='hann', center=True, pad_mode='reflect', power=2.0),
    PowerToDbTransform(ref=np.max, amin=1e-10, top_db=80.0),
    StandardizationTransform(),
    ExpandDimsTransform(axis=0),
    ToTensorTransform()
])

In [131]:
x_processed = transform(x_stretch)

d:\Projects\asp-azhmr\src\dataset_v5.py:191: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(torch.from_numpy(y),


In [132]:
x_processed.shape

torch.Size([1, 128, 431])

In [133]:
x_toDevice = x_processed.to('cuda')

In [134]:
y_pred = model(x_toDevice)

In [135]:
sd.play(x)

In [136]:
y_pred

tensor([[1.0000e+00, 5.0944e-12]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [137]:
y_pred.argmax(1).cpu().numpy()[0]

0